In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

fine_tuning_lm_physical_interpretation_hackathon_path = kagglehub.competition_download('fine-tuning-lm-physical-interpretation-hackathon')

print('Data source import complete.')


100%|██████████| 296k/296k [00:00<00:00, 859kB/s]

Extracting files...
Data source import complete.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!pip install pyvista transformers datasets peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [1]:
!pip install pyvista transformers datasets peft accelerate

import numpy as np
import pyvista as pv
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
import torch
import gc
import os
from accelerate import Accelerator

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

accelerator = Accelerator(cpu=True)

def solve_heat_equation_case1():
    nx, ny = 50, 50  
    dx, dy = 1 / (nx - 1), 1 / (ny - 1) 
    x = np.linspace(0, 1, nx)
    y = np.linspace(0, 1, ny)
    X, Y = np.meshgrid(x, y)

    T = np.zeros((ny, nx))

    T[0, :] = 0  
    T[-1, :] = 0 
    T[:, 0] = 0  
    T[:, -1] = 0 

    def force_function(x, y):
        return 8 * np.pi**2 * np.sin(2 * np.pi * x) * np.sin(2 * np.pi * y)

    for _ in range(1000):  
        T_new = T.copy()
        for i in range(1, nx - 1):
            for j in range(1, ny - 1):
                T_new[j, i] = 0.25 * (T[j + 1, i] + T[j - 1, i] + T[j, i + 1] + T[j, i - 1] - dx**2 * force_function(x[i], y[j]))
        T = T_new

    grid = pv.StructuredGrid(X, Y, np.zeros_like(X))
    grid["Temperature"] = T.ravel()
    grid.save("Case1_FDM.vtk")

solve_heat_equation_case1()

grid = pv.read("Case1_FDM.vtk")

# Extract data
points = grid.points
temperature = grid["Temperature"]

data = pd.DataFrame({
    "x": points[:, 0],
    "y": points[:, 1],
    "temperature": temperature
})

data.to_csv("Case1_FDM.csv", index=False)


model_name = "ibm-granite/granite-3.1-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model.gradient_checkpointing_enable()

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=4, 
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

data = pd.read_csv("Case1_FDM.csv")
dataset = Dataset.from_pandas(data)

def preprocess_function(examples):
    examples["temperature"] = [str(temp) for temp in examples["temperature"]]
    tokenized_inputs = tokenizer(examples["temperature"], truncation=True, max_length=512)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()

    return tokenized_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

torch.cuda.empty_cache()
gc.collect()

training_args = TrainingArguments(
    output_dir="./results_lora",
    per_device_train_batch_size=1,  
    gradient_accumulation_steps=8,  
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,  
    save_strategy="epoch", 
    logging_dir="./logs",  
    logging_steps=10,  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,  
)

model, trainer = accelerator.prepare(model, trainer)

trainer.train()


questions = [
    "What is the temperature distribution at the corner (0, 0) of the unit square mesh?",
    "How does the temperature change with respect to the position along the x-axis at y = 0.5?",
    "If we increase the coefficient of pi in the force function, what will happen?",
    "Explain why the temperature is zero at both x=0 and x=1, and what this means physically.",
    "At what coordinates does the maximum temperature occur, and what determines this location?",
    "How does the temperature profile change along the vertical line x=0.5 compared to x=0.25?",
    "What is the temperature at the corner (0, 0) of the unit square mesh?",
    "What physical significance does the boundary condition T(0,y)=0 have in this context?",
    "What does the boundary condition T(1,y)=y(1-y) represent physically in this problem?",
    "What can you infer about the decay rate of temperature?",
    "Comment on the physical interpretation of why the spatial pattern remains unchanged while only the amplitude decreases with time?",
    "What is the effect of alpha on the decay rate of heat dissipation?"
]

responses = []
for question in questions:
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=100)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    responses.append(response)

submission = pd.DataFrame({
    "ID": range(1, 13),
    "Answer": responses
})

# Save to CSV
submission.to_csv("submission.csv", index=False)

print("Submission file created successfully!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sallarohit10 (sallarohit10-srm-institute-of-science-and-technology). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,23.080400
20,22.979100
30,23.386900
40,22.257600
50,22.036100
60,21.581400
70,21.104200
80,20.165800
90,20.343500
100,20.064600


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2134: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [2]:

questions = [
    "What is the temperature distribution at the corner (0, 0) of the unit square mesh?",
    "How does the temperature change with respect to the position along the x-axis at y = 0.5?",
    "If we increase the coefficient of pi in the force function, what will happen?",
    "Explain why the temperature is zero at both x=0 and x=1, and what this means physically.",
    "At what coordinates does the maximum temperature occur, and what determines this location?",
    "How does the temperature profile change along the vertical line x=0.5 compared to x=0.25?",
    "What is the temperature at the corner (0, 0) of the unit square mesh?",
    "What physical significance does the boundary condition T(0,y)=0 have in this context?",
    "What does the boundary condition T(1,y)=y(1-y) represent physically in this problem?",
    "What can you infer about the decay rate of temperature?",
    "Comment on the physical interpretation of why the spatial pattern remains unchanged while only the amplitude decreases with time?",
    "What is the effect of alpha on the decay rate of heat dissipation?"
]

responses = []
for question in questions:
    inputs = tokenizer(question, return_tensors="pt")

    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    outputs = model.generate(**inputs, max_length=100)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    responses.append(response)

submission = pd.DataFrame({
    "ID": range(1, 13),
    "Answer": responses
})

submission.to_csv("submission.csv", index=False)

print("Submission file created successfully!")

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Submission file created successfully!
